In [1]:
from datetime import date, datetime, timedelta, time
import os
import pytz
import json
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
%cd desktop/metrics-mvp/models

C:\Users\jose\desktop\metrics-mvp\models


In [4]:
all_files = []

In [6]:
filepaths = [f"..\\data\\timetables\\{filename}" for filename in os.listdir("..\\data\\timetables")]

In [7]:
for fp in filepaths:
    all_files.append(pd.read_csv(fp))

In [36]:
all_stops_timetables = {df["ROUTE"][0]:{stop_id:df[df["stop_id"] == stop_id] for stop_id in df["stop_id"].unique()} for df in all_files}

In [12]:
all_stops_timetables.keys()

dict_keys([10, 12, '14R', '14X', 14, 18, 19, '1AX', '1BX', 1, 21, 22, 23, 24, 25, 27, '28R', 28, 29, 2, '30X', 30, '31AX', '31BX', 31, 33, 35, 36, 37, '38AX', '38BX', '38R', 38, 39, 3, 41, 43, 44, 45, 47, 48, 49, 52, 54, 55, 56, 57, '5R', 5, 66, 67, 6, 714, '76X', '7X', 7, '82X', 88, '8AX', '8BX', 8, 90, 91, '9R', 9, 'C', 'F', 'J', 'KT', 'L-OWL', 'L', 'M-OWL', 'M', 'N-OWL', 'NX', 'N', 'PH', 'PM', 'S', 'T-OWL', 'TBUS'])

In [14]:
all_stops_timetables[10].keys()

dict_keys([17518, 14350, 16700, 16695, 17549, 16333, 15852, 15850, 15147, 15853, 16327, 13011, 16694, 16701, 13313, 15859])

In [37]:
test = all_stops_timetables[10][17518]

In [38]:
test.head()

,ROUTE,direction,timetable,trip_id,stop_id,TIME
0,10,IB,Saturday,8528062,17518,2019-03-21 06:35:00
8,10,IB,Saturday,8528063,17518,2019-03-21 06:55:00
16,10,IB,Saturday,8528064,17518,2019-03-21 07:15:00
24,10,IB,Saturday,8528065,17518,2019-03-21 07:35:00
32,10,IB,Saturday,8528066,17518,2019-03-21 07:55:00


In [19]:
tz = pytz.timezone('US/Pacific')

In [25]:
test.timetable.unique()

array(['Saturday', 'Sunday', 'Weekday'], dtype=object)

In [39]:
types = [test[test["timetable"] == day_type] for day_type in ["Saturday", "Sunday", "Weekday"]]

In [42]:
for t in types:
    t["DATE_TIME"] = t["TIME"].apply(lambda x: tz.localize(datetime.strptime(x, "%Y-%m-%d %H:%M:%S")))
    t["headway"] = t["DATE_TIME"] - t["DATE_TIME"].shift(1)
    print(t["headway"].describe())

C:\Users\jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count                          95
mean     -1 days +23:56:05.684210
std        0 days 02:59:26.309414
min             -1 days +00:30:00
25%               0 days 00:20:00
50%               0 days 00:20:00
75%               0 days 00:21:00
max               0 days 00:30:00
Name: headway, dtype: object
count                          95
mean     -1 days +23:56:05.684210
std        0 days 02:59:26.309414
min             -1 days +00:30:00
25%               0 days 00:20:00
50%               0 days 00:20:00
75%               0 days 00:21:00
max               0 days 00:30:00
Name: headway, dtype: object
count                         135
mean     -1 days +23:57:54.666666
std        0 days 02:31:01.380145
min             -1 days +00:30:00
25%               0 days 00:14:00
50%               0 days 00:15:00
75%               0 days 00:16:00
max               0 days 00:30:00
Name: headway, dtype: object


In [43]:
print([t.tail() for t in types])

[      ROUTE direction timetable  trip_id  stop_id                 TIME  \
1752     10        OB  Saturday  8528056    17518  2019-03-21 22:25:00   
1760     10        OB  Saturday  8528057    17518  2019-03-21 22:54:00   
1768     10        OB  Saturday  8528058    17518  2019-03-21 23:24:00   
1776     10        OB  Saturday  8528059    17518  2019-03-21 23:54:00   
1784     10        OB  Saturday  8528060    17518  2019-03-21 00:24:00   

                     DATE_TIME           headway  
1752 2019-03-21 22:25:00-07:00          00:30:00  
1760 2019-03-21 22:54:00-07:00          00:29:00  
1768 2019-03-21 23:24:00-07:00          00:30:00  
1776 2019-03-21 23:54:00-07:00          00:30:00  
1784 2019-03-21 00:24:00-07:00 -1 days +00:30:00  ,       ROUTE direction timetable  trip_id  stop_id                 TIME  \
2136     10        OB    Sunday  8528152    17518  2019-03-21 22:25:00   
2144     10        OB    Sunday  8528153    17518  2019-03-21 22:54:00   
2152     10        OB    

In [45]:
day_timedelta = timedelta(days = -1)

In [48]:
types[0][types[0]["DATE_TIME"] == types[0]["DATE_TIME"].max()]

,ROUTE,direction,timetable,trip_id,stop_id,TIME,DATE_TIME,headway
1776,10,OB,Saturday,8528059,17518,2019-03-21 23:54:00,2019-03-21 23:54:00-07:00,00:30:00


In [60]:
len(types[0])

96

In [63]:
for t in types:
    t.index = range(len(t))

In [96]:
for t in types:
    latest_stop_of_day = t["DATE_TIME"].idxmax()
    
    for idx, row in t.iterrows():
        if idx > latest_stop_of_day:
            

ROUTE                               10
direction                           OB
timetable                     Saturday
trip_id                        8528060
stop_id                          17518
TIME               2019-03-21 00:24:00
DATE_TIME    2019-03-21 00:24:00-07:00
headway              -1 days +00:30:00
Name: 95, dtype: object
ROUTE                               10
direction                           OB
timetable                       Sunday
trip_id                        8528156
stop_id                          17518
TIME               2019-03-21 00:24:00
DATE_TIME    2019-03-21 00:24:00-07:00
headway              -1 days +00:30:00
Name: 95, dtype: object
ROUTE                               10
direction                           OB
timetable                      Weekday
trip_id                        8527887
stop_id                          17518
TIME               2019-03-21 00:23:00
DATE_TIME    2019-03-21 00:23:00-07:00
headway              -1 days +00:30:00
Name: 135, dtype

In [95]:
for t in types:
    print(t.tail())

    ROUTE direction timetable  trip_id  stop_id                 TIME  \
91     10        OB  Saturday  8528056    17518  2019-03-21 22:25:00   
92     10        OB  Saturday  8528057    17518  2019-03-21 22:54:00   
93     10        OB  Saturday  8528058    17518  2019-03-21 23:24:00   
94     10        OB  Saturday  8528059    17518  2019-03-21 23:54:00   
95     10        OB  Saturday  8528060    17518  2019-03-21 00:24:00   

                   DATE_TIME           headway  
91 2019-03-21 22:25:00-07:00          00:30:00  
92 2019-03-21 22:54:00-07:00          00:29:00  
93 2019-03-21 23:24:00-07:00          00:30:00  
94 2019-03-21 23:54:00-07:00          00:30:00  
95 2019-03-21 00:24:00-07:00 -1 days +00:30:00  
    ROUTE direction timetable  trip_id  stop_id                 TIME  \
91     10        OB    Sunday  8528152    17518  2019-03-21 22:25:00   
92     10        OB    Sunday  8528153    17518  2019-03-21 22:54:00   
93     10        OB    Sunday  8528154    17518  2019-03-

In [76]:
types[0]["DATE_TIME"]

Timestamp('2019-03-21 00:24:00-0700', tz='US/Pacific')

In [98]:
types[0].iloc[95]["adjusted_time"] = types[0].iloc[95]["DATE_TIME"] + timedelta(days = 1)

C:\Users\jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\jose\Anaconda3\lib\site-packages\pandas\core\series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [99]:
types[0].tail()

,ROUTE,direction,timetable,trip_id,stop_id,TIME,DATE_TIME,headway
91,10,OB,Saturday,8528056,17518,2019-03-21 22:25:00,2019-03-21 22:25:00-07:00,00:30:00
92,10,OB,Saturday,8528057,17518,2019-03-21 22:54:00,2019-03-21 22:54:00-07:00,00:29:00
93,10,OB,Saturday,8528058,17518,2019-03-21 23:24:00,2019-03-21 23:24:00-07:00,00:30:00
94,10,OB,Saturday,8528059,17518,2019-03-21 23:54:00,2019-03-21 23:54:00-07:00,00:30:00
95,10,OB,Saturday,8528060,17518,2019-03-21 00:24:00,2019-03-21 00:24:00-07:00,-1 days +00:30:00


In [107]:
types[0].tail()

,ROUTE,direction,timetable,trip_id,stop_id,TIME,DATE_TIME,headway,headway_from_max
91,10,OB,Saturday,8528056,17518,2019-03-21 22:25:00,2019-03-21 22:25:00-07:00,00:30:00,-1 days +22:31:00
92,10,OB,Saturday,8528057,17518,2019-03-21 22:54:00,2019-03-21 22:54:00-07:00,00:29:00,-1 days +23:00:00
93,10,OB,Saturday,8528058,17518,2019-03-21 23:24:00,2019-03-21 23:24:00-07:00,00:30:00,-1 days +23:30:00
94,10,OB,Saturday,8528059,17518,2019-03-21 23:54:00,2019-03-21 23:54:00-07:00,00:30:00,00:00:00
95,10,OB,Saturday,8528060,17518,2019-03-21 00:24:00,2019-03-21 00:24:00-07:00,-1 days +00:30:00,-1 days +00:30:00


In [109]:
test_type = types[0]

In [110]:
test_type["idx"] = test_type.index

C:\Users\jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [111]:
test_type

,ROUTE,direction,timetable,trip_id,stop_id,TIME,DATE_TIME,headway,headway_from_max,idx
0,10,IB,Saturday,8528062,17518,2019-03-21 06:35:00,2019-03-21 06:35:00-07:00,NaT,-1 days +06:41:00,0
1,10,IB,Saturday,8528063,17518,2019-03-21 06:55:00,2019-03-21 06:55:00-07:00,00:20:00,-1 days +07:01:00,1
2,10,IB,Saturday,8528064,17518,2019-03-21 07:15:00,2019-03-21 07:15:00-07:00,00:20:00,-1 days +07:21:00,2
3,10,IB,Saturday,8528065,17518,2019-03-21 07:35:00,2019-03-21 07:35:00-07:00,00:20:00,-1 days +07:41:00,3
4,10,IB,Saturday,8528066,17518,2019-03-21 07:55:00,2019-03-21 07:55:00-07:00,00:20:00,-1 days +08:01:00,4
5,10,IB,Saturday,8528067,17518,2019-03-21 08:15:00,2019-03-21 08:15:00-07:00,00:20:00,-1 days +08:21:00,5
6,10,IB,Saturday,8528068,17518,2019-03-21 08:35:00,2019-03-21 08:35:00-07:00,00:20:00,-1 days +08:41:00,6
7,10,IB,Saturday,8528069,17518,2019-03-21 08:55:00,2019-03-21 08:55:00-07:00,00:20:00,-1 days +09:01:00,7
8,10,IB,Saturday,8528070,17518,2019-03-21 09:10:00,2019-03-21 09:10:00-07:00,00:15:00,-1 days +09:16:00,8
9,10,IB,Saturday,8528073,17518,2019-03-21 09:30:00,2019-03-21 09:30:00-07:00,00:20:00,-1 days +09:36:00,9


In [118]:
test_type.loc[(test_type["idx"] > 94), ["DATE_TIME"]] = test_type.loc[(test_type["idx"] > 94), ["DATE_TIME"]] + timedelta(days = 1)

In [119]:
test_type.loc[(test_type["idx"] > 94), ["DATE_TIME"]]

,DATE_TIME
95,2019-03-22 00:24:00-07:00


In [120]:
test_type["TIME"].apply(lambda x: x.split(" "))

0     [2019-03-21, 06:35:00]
1     [2019-03-21, 06:55:00]
2     [2019-03-21, 07:15:00]
3     [2019-03-21, 07:35:00]
4     [2019-03-21, 07:55:00]
5     [2019-03-21, 08:15:00]
6     [2019-03-21, 08:35:00]
7     [2019-03-21, 08:55:00]
8     [2019-03-21, 09:10:00]
9     [2019-03-21, 09:30:00]
10    [2019-03-21, 09:50:00]
11    [2019-03-21, 10:10:00]
12    [2019-03-21, 10:30:00]
13    [2019-03-21, 10:50:00]
14    [2019-03-21, 11:10:00]
15    [2019-03-21, 11:30:00]
16    [2019-03-21, 11:50:00]
17    [2019-03-21, 12:10:00]
18    [2019-03-21, 12:30:00]
19    [2019-03-21, 12:50:00]
20    [2019-03-21, 13:10:00]
21    [2019-03-21, 13:30:00]
22    [2019-03-21, 13:50:00]
23    [2019-03-21, 14:10:00]
24    [2019-03-21, 14:30:00]
25    [2019-03-21, 14:50:00]
26    [2019-03-21, 15:10:00]
27    [2019-03-21, 15:30:00]
28    [2019-03-21, 15:50:00]
29    [2019-03-21, 16:10:00]
               ...          
66    [2019-03-21, 13:17:00]
67    [2019-03-21, 13:37:00]
68    [2019-03-21, 13:57:00]
69    [2019-03

In [122]:
test_type.index > 94

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True])